In [1]:
import sys
# sys.path
import time
import ee
import pprint
from IPython.display import Image
# from image import Image as local_model
import openet.ssebop as local_model

In [2]:
try:
    ee.Initialize()
    print('worked')
except ee.EEException:
    ee.Authenticate()
    ee.Initialize()
    print('worked second try')

worked


In [3]:
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
    'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
    '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']
image_size = 768

In [4]:
def ee_task_start(task, n=10):
    """Make an exponential backoff Earth Engine request"""
    output = None
    for i in range(1, n):
        try:
            task.start()
            break
        except Exception as e:
            time.sleep(i ** 2)
    return task

In [5]:

# # DEADBEEF
# image_names = ['LC08_L2TP_035035_20170615_20170701', 'LC08_L2TP_035035_20170701_20170717', 'LE07_L2TP_035035_20170709_20170804',
#                'LC08_L2TP_035035_20180602_20180615', 'LC08_L2TP_035035_20180618_20180704', 'LE07_L2TP_035035_20180728_20180824', 
#                'LE07_L2TP_035035_20190613_20190710', 'LC08_L2TP_035035_20190621_20190706', 'LC08_L2TP_035035_20190824_20190903']
# image_paths = []
# for img in image_names:
#     img_lst = img.split('_')
#     root = 'LANDSAT/{}/C01/T1_TOA'.format(img_lst[0])
#     img_name = f'{img_lst[0]}_{img_lst[2]}_{img_lst[3]}'
#     img_path = f'{root}/{img_name}'
#     image_paths.append(img_path)

# alabama: LC08_020037_20150124, Alpharetta: LC08_019036_20150101, WV: LE07_019034_20150109, Jonesboro: LE07_023035_20150105
# TODO: add the ingest date on here somehow.
image_names = ['LC08_L2TP_020037_20150124', 'LC08_L2TP_019036_20150101','LE07_L2TP_019034_20150109', 'LE07_L2TP_023035_20150105']
image_paths = []
for img in image_names:
    img_lst = img.split('_')
    root = 'LANDSAT/{}/C01/T1_TOA'.format(img_lst[0])
    img_name = f'{img_lst[0]}_{img_lst[2]}_{img_lst[3]}'
    img_path = f'{root}/{img_name}'
    image_paths.append(img_path)


In [6]:
print(image_paths)

['LANDSAT/LC08/C01/T1_TOA/LC08_020037_20150124', 'LANDSAT/LC08/C01/T1_TOA/LC08_019036_20150101', 'LANDSAT/LE07/C01/T1_TOA/LE07_019034_20150109', 'LANDSAT/LE07/C01/T1_TOA/LE07_023035_20150105']


In [7]:

# TODO - FIXXXXX

for n, p in zip(image_names, image_paths):
    # landsat images
    print(f'doing {n}')
    landsat_img = ee.Image(p)
    landsat_crs = landsat_img.select('B3').projection().getInfo()['crs']
    landsat_region = landsat_img.geometry().bounds(1, 'EPSG:4326')
    landsat_dt = ee.Date(landsat_img.get('system:time_start'))
    # Build the SSEBop object from the Landsat image
    test = local_model.Image(None)
    print(test.salutations)
    model_obj = local_model.Image.from_landsat_c1_toa(
        landsat_img,
        et_reference_source='IDAHO_EPSCOR/GRIDMET', 
        et_reference_band='etr', 
        et_reference_factor=0.85,
        et_reference_resample='nearest',
        tcorr_source='GRIDDED',
    )
    # Initiate the Lazy Attributes
    tcorr = model_obj.tcorr_gridded
    ETf = model_obj.et_fraction
    
    attempt = 'I'
    level = n
    
    change = 'ColdPixelCountAndKernelShape'
#     change = 'ColdPixelCount'
#     change = 'Original'
    name_root = f'{level}_vers{attempt}_changed{change}'
    tcorr_name =  f'tcorr_{name_root}'
    etf_name = f'etf_{name_root}'
    eta_name = f'eta_{name_root}'
    
    task = ee.batch.Export.image.toAsset(image=tcorr , description=tcorr_name, assetId=f'users/gbrlparrish/gridCv4Dev/{tcorr_name}', region=landsat_region)
#     task = ee.batch.Export.image.toDrive(image=tcorr,  description=tcorr_name, region=landsat_region)
    # start the tasktest
    print('starting tcorr export')
    ee_task_start(task=task)
    print('task submitted')
    task = ee.batch.Export.image.toAsset(image=ETf, description=etf_name, assetId=f'users/gbrlparrish/gridCv4Dev/{etf_name}', region=landsat_region)
#     task = ee.batch.Export.image.toDrive(image=ETf,  description=etf_name, region=landsat_region)
    
    # start the task
    print('starting ETf export')
    ee_task_start(task=task)
    print('task submitted')

doing LC08_L2TP_020037_20150124


AttributeError: 'NoneType' object has no attribute 'projection'

In [ ]:

# # matching ETo image
# eto_img = ee.Image('projects/usgs-ssebop/pet/gridmet_median_v1/doy196')


In [ ]:
# ndvi_image = model_obj.ndvi
# print(ndvi_image.getInfo())

In [ ]:
# # === USER ====
# attempt = 'I'
# level = 'GS_Alg'
# name_root = f'rn{level}_vers{attempt}'

# tcorr_name =  f'a_test_tcorr_{name_root}'
# etf_name = f'a_test_etf_{name_root}'
# eta_name = f'a_test_eta_{name_root}'

In [ ]:
# # try to export and see if the image looks like expected.
# # task = ee.batch.Export.image.toDrive(image=tcorr_image, description='CFact_TestGTE', region=landsat_region)
# task = ee.batch.Export.image.toAsset(image=tcorr , description=tcorr_name, assetId=f'users/gbrlparrish/gridCForMattS/{tcorr_name}', region=landsat_region)
# # start the tasktest
# print('starting')
# ee_task_start(task=task)
# print('task submitted')
# # task2 = ee.batch.Export.image.toAsset(image= tcorr_rn02_hot_testI, description='tcorr_rn02_hot_testI', assetId='users/gbrlparrish/griddedC/tcorr_rn02_hot_testI', region=landsat_region)
# # # start the task
# # print('starting task 2')
# # ee_task_start(task=task2)
# # print('task 2 submitted')
# # # # 

In [ ]:
# ETf = model_obj.et_fraction
# # make ETa by multiplying by gridmet ETo v1
# ETa = ETf.multiply(eto_img)

In [ ]:
# # try to export EToF (Doesn't work. Error is "Error: Too many input pixels per output pixel. Need 27778, but only 64 allowed." Like Whaaaaat)
# task = ee.batch.Export.image.toAsset(image=ETf, description=etf_name, assetId=f'users/gbrlparrish/griddedC/{etf_name}', region=landsat_region)
# # start the task
# print('starting')
# ee_task_start(task=task)
# print('task submitted')
# # ============================
# task = ee.batch.Export.image.toAsset(image=ETa, description=eta_name, assetId=f'users/gbrlparrish/griddedC/{eta_name}', region=landsat_region)
# # start the task
# print('starting')
# ee_task_start(task=task)
# print('task submitted')

# # # TODO - export as asset and then display in the notebook.
# # # # # =========================================================================
# # # # # Try to show EToF and it also doesn't work
# # # image_url = model_obj.et_fraction\
# # #     .getThumbURL({'min': 0, 'max': 1.2, 'palette': et_palette, 
# # #                   'region': landsat_region, 'dimensions': image_size})
# # # Image(image_url, embed=True, format='png')